In [1]:
# Import packages
import sys

sys.path.append("../")

from db import models
from pathlib import Path
import os
from run import get_meta, get_closest
import cv2
import matplotlib.pyplot as plt
import exifread
import math
from math import radians, cos
from shapely import wkt
from PIL import Image
import yaml
from src.azure_blobs import download_image, get_blobs_by_folder_name
from tqdm import tqdm
from pyproj import Proj, transform
import numpy as np
import pickle 

IMAGE_FORMATS = (".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG")

In [2]:
# Load inspection points from MSS
try:
    points = models.InspectionPoints.get_all()
except Exception as e:
    print(e)

(pyodbc.InterfaceError) ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')
(Background on this error at: https://sqlalche.me/e/20/rvf5)


In [3]:
# Reading the input points 
points = pickle.load(open("input_points/points.pkl", "rb"))

In [4]:
# Define current path
current_path = Path().absolute()

In [5]:
# Reading the configuration.yml file
with open(os.path.join(current_path,'..', "configuration.yml"), "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [6]:
images_input_dir = os.path.join(current_path, "input")
os.makedirs(images_input_dir, exist_ok=True)

In [20]:
images_output_dir = os.path.join(current_path, "output")
os.makedirs(images_output_dir, exist_ok=True)

In [8]:
storage_folder_name = (
    "RnD/Gatvių valymas/2023-2024/20231130 Ateieties-Jaruzales-saligatviai/"
)

In [17]:
storage_images = get_blobs_by_folder_name(
    config=config, name_starts_with=storage_folder_name
)
# Downloading images from Azure storage to local dir
for img in tqdm(storage_images, desc="Downloading images from Azure Storage:"):
    download_image(blob_name=img, config=config, local_file_dir=images_input_dir)

Getting current blobs: 33it [00:00, 34.51it/s]


In [9]:
images = []
for root, dirs, files in os.walk(images_input_dir):
    for file in files:
        # Infering whether the file ends with .jpg, .JPG, .jpeg, .png, .PNG
        if file.endswith(IMAGE_FORMATS):
            # Adding the full path to the file
            file = os.path.join(root, file)
            # Appending to the list of images to infer
            images.append(file)

In [10]:
# Extracting the function to extract the focal length
def extract_focal_length(file_path):
    # Open the image file for reading in binary mode
    with open(file_path, "rb") as f:
        # Read the EXIF data
        tags = exifread.process_file(f)

        # Check if GPSInfo tag is present
        if "EXIF FocalLength" in tags:
            # Extract latitude, longitude, and altitude
            focal_length = tags["EXIF FocalLength"].values[0]

            return float(focal_length)
        else:
            print("Focal length not found in the metadata.")
            return None

In [11]:
def extract_gps_coordinates(file_path):
    # Open the image file for reading in binary mode
    with open(file_path, "rb") as f:
        # Read the EXIF data
        tags = exifread.process_file(f)

        # Check if GPSInfo tag is present
        if (
            "GPS GPSLatitude" in tags
            and "GPS GPSLongitude" in tags
            and "GPS GPSAltitude" in tags
        ):
            # Extract latitude, longitude, and altitude
            latitude = tags["GPS GPSLatitude"].values
            longitude = tags["GPS GPSLongitude"].values
            altitude = tags["GPS GPSAltitude"].values

            # Convert coordinates to decimal format
            latitude_decimal = latitude[0] + latitude[1] / 60 + latitude[2] / 3600
            longitude_decimal = longitude[0] + longitude[1] / 60 + longitude[2] / 3600

            return float(latitude_decimal), float(longitude_decimal), float(altitude[0])
        else:
            print("GPS information not found in the metadata.")
            return None

In [12]:
# Defining the function that calculates the diff in pixels to meters
# GSD - Ground Sampling Distance
def calculate_gsd(
    height_from_ground,
    image_width,
    image_height,
    sensor_width,
    sensor_height,
    focal_length,
):
    """
    Function that calculates the GSD (Ground Sampling Distance) from pixels to meters

    Args:
        height_from_ground (float): Height from ground in meters
        image_width (int): Image width in pixels
        image_height (int): Image height in pixels
        sensor_width (float): Sensor width in mm
        sensor_height (float): Sensor height in mm
        focal_length (float): Focal length in mm

    Returns:
        gsd_h (float): Horizontal GSD in meters
        gsd_v (float): Vertical GSD in meters
    """
    # Calculating the horizontal and vertical GSD
    gsd_h = (height_from_ground * sensor_width) / (focal_length * image_width)
    gsd_v = (height_from_ground * sensor_height) / (focal_length * image_height)
    # Returning the average GSD
    return gsd_h, gsd_v

In [13]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers

    # Convert degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Differences in coordinates
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad

    # Haversine formula
    a = (
        math.sin(delta_lat / 2) ** 2
        + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(delta_lon / 2) ** 2
    )

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c

    return distance


def calculate_distances(point_A, point_B):
    x_distance = haversine_distance(
        (point_A[0] + point_B[0]) / 2,
        point_A[1],
        (point_A[0] + point_B[0]) / 2,
        point_B[1],
    )

    y_distance = haversine_distance(
        point_A[0],
        (point_A[1] + point_B[1]) / 2,
        point_B[0],
        (point_A[1] + point_B[1]) / 2,
    )

    return x_distance, y_distance

In [14]:
def flat_earth_coordinates_difference(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Calculate differences in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Calculate horizontal (x) and vertical (y) differences
    horizontal_difference = dlon * cos(lat1)
    vertical_difference = dlat

    # Assuming Earth's radius is 6371 km
    earth_radius = 6371.0

    # Convert differences to meters
    horizontal_difference_meters = horizontal_difference * earth_radius * 1000
    vertical_difference_meters = vertical_difference * earth_radius * 1000

    return horizontal_difference_meters, vertical_difference_meters

In [15]:
def lonlat_to_utm(lon, lat):
    utm_projection = Proj(proj="utm", zone=33, ellps="WGS84", preserve_units=False)
    return transform(Proj(init="epsg:4326"), utm_projection, lon, lat)


def calculate_pixel_difference(lon1, lat1, lon2, lat2, gsd_x, gsd_y):
    x1, y1 = lonlat_to_utm(lon1, lat1)
    x2, y2 = lonlat_to_utm(lon2, lat2)

    distance_x = np.abs(x2 - x1)
    distance_y = np.abs(y2 - y1)

    pixel_diff_x = distance_x / gsd_x
    pixel_diff_y = distance_y / gsd_y

    return pixel_diff_x, pixel_diff_y

In [16]:
# Define sensor width and sensor height
sensor_width = 10.16
sensor_height = 7.619999999999999

In [22]:
for _img in tqdm(images):
    # Get image center coordinates
    x, y = get_meta(_img)
    # Get closest inspection point
    closest_point = get_closest((x, y), points)
    # Open image with OpenCV
    img = Image.open(_img)
    img = np.array(img)
    img = img[:, :, ::-1].copy()
    # Get image height and width
    h, w = img.shape[0], img.shape[1]
    # Get rectangle with 25 pixel offset
    center_box = [w / 2 - 25, h / 2 - 25, w / 2 + 25, h / 2 + 25]
    center_box = [int(x) for x in center_box]
    #  Draw rectange on image center
    img = cv2.rectangle(
        img,
        (center_box[0], center_box[1]),
        (center_box[2], center_box[3]),
        (255, 0, 0),
        2,
    )
    # Get focal lenght from image metadata
    focal_length = extract_focal_length(_img)
    # Get image center coordinate and absoulute height
    y, x, z = extract_gps_coordinates(_img)
    # Load inspection point geometry
    point = wkt.loads(closest_point["Shape"])
    # Get xmp metadata from image
    xmp = Image.open(_img).getxmp()
    # Get relative height
    rel_alt = xmp["xmpmeta"]["RDF"]["Description"]["RelativeAltitude"]
    rel_alt = float(rel_alt)
    # Calculate gsd values
    gsd_h, gsd_v = calculate_gsd(
        rel_alt, w, h, sensor_width, sensor_height, focal_length
    )
    horizontal_distance, vertical_distance = flat_earth_coordinates_difference(
        y, x, point.y, point.x
    )

    # Convert meters to pixels
    horizontal_pixels = horizontal_distance / gsd_v
    vertical_pixels = vertical_distance / gsd_h

    # Get rectangle with 25 pixel offset
    addjusted_box = [
        (w / 2 - 25) + vertical_pixels,
        (h / 2 - 25) + horizontal_pixels,
        (w / 2 + 25) + vertical_pixels,
        (h / 2 + 25) + horizontal_pixels,
    ]
    addjusted_box = [int(x) for x in addjusted_box]

    #  Draw rectange on image center
    img = cv2.rectangle(
        img,
        (addjusted_box[0], addjusted_box[1]),
        (addjusted_box[2], addjusted_box[3]),
        (0, 0, 255),
        2,
    )

    cv2.imwrite(os.path.join(images_output_dir, os.path.basename(_img)), img)

100%|██████████| 33/33 [00:36<00:00,  1.11s/it]


: 

In [18]:
os.path.join(images_output_dir, os.path.basename(_img))

'c:\\Users\\Admin\\projects\\snow-identifier\\notebooks\\output\\DJI_0998-2023-11-30-11-25-18.jpg'